<a href="https://colab.research.google.com/github/soohyoen/artificial-intelligence/blob/main/Fine_tuning_BERT_for_sentiment_class_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT for sentiment analysis
- author: Eu-Bin KIM
- date: 6th of October 2021

## Contents
1. 학습데이터 확인
2. 보조함수 확인
3. SenitmentClassifier 확인
4. Analyser 확인
5. 학습 과정 확인
6. tests

## 학습 데이터

In [ ]:
!pip3 install torch
!pip3 install transformers
from typing import List, Tuple
from transformers import BertTokenizer, BertModel
import torch
from torch.nn import functional as F


DATA: List[Tuple[str, int]] = [
    # 긍정적인 문장 - 1
    ("도움이 되었으면", 1),
    # 병국님
    ("오늘도 수고했어", 1),
    # 영성님
    ("너는 할 수 있어", 1),
    # 정무님
    ("오늘 내 주식이 올랐다", 1),
    # 우철님
    ("오늘 날씨가 좋다", 1),
    # 유빈님
    ("난 너를 좋아해", 1),
    # 다운님
    ("지금 정말 잘하고 있어", 1),
    # 민종님
    ("지금처럼만 하면 잘될거야", 1),
    ("사랑해", 1),
    ("저희 허락없이 아프지 마세요", 1),
    ("오늘 점심 맛있다", 1),
    ("오늘 너무 예쁘다", 1),
    # 다운님
    ("곧 주말이야", 1),
    # 재용님
    ("오늘 주식이 올랐어", 1),
    # 병운님
    ("우리에게 빛나는 미래가 있어", 1),
    # 재용님
    ("너는 참 잘생겼어", 1),
    # 윤서님
    ("콩나물 무침은 맛있어", 1),
    # 정원님
    ("강사님 보고 싶어요", 1),
    # 정원님
    ("오늘 참 멋있었어", 1),
    # 예은님
    ("맛있는게 먹고싶다", 1),
    # 민성님
    ("로또 당첨됐어", 1),
    # 민성님
    ("이 음식은 맛이 없을수가 없어", 1),
    # 경서님
    ("오늘도 좋은 하루보내요", 1),
    # 성민님
    ("내일 시험 안 본대", 1),
    # --- 부정적인 문장 - 레이블 = 0
    ("난 너를 싫어해", 0),
    # 병국님
    ("넌 잘하는게 뭐냐?", 0),
    # 선희님
    ("너 때문에 다 망쳤어", 0),
    # 정무님
    ("오늘 피곤하다", 0),
    # 유빈님
    ("난 삼성을 싫어해", 0),
    ("진짜 가지가지 한다", 0),
    ("꺼져", 0),
    ("그렇게 살아서 뭘하겠니", 0),
    # 재용님 - 주식이 파란불이다?
    ("오늘 주식이 파란불이야", 0),
    # 지현님
    ("나 오늘 예민해", 0),
    ("주식이 떨어졌다", 0),
    ("콩나물 다시는 안먹어", 0),
    ("코인 시즌 끝났다", 0),
    ("배고파 죽을 것 같아", 0),
    ("한강 몇도냐", 0),
    ("집가고 싶다", 0),
    ("내 미래가 흐리다", 0), 
    ("나 보기가 역겨워", 0),  # 긍정적인 확률이 0
    # 진환님
    ("나는 너가 싫어", 0),
    ("잘도 그러겠다", 0),
    ("너는 어렵다", 0),
    # ("자연어처리는", 0)
]

## 보조함수

In [ ]:
# cuda를 사용할 수 있는지를 체크, 사용가능하다면 cuda로 설정된 device를 출력.
def load_device() -> torch.device:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return device


# 텐서를 구축하는 부분 - X
def build_X(sents: List[str], tokenizer: BertTokenizer, device: torch.device) -> torch.Tensor:
    """
    return: X (N, 3, L).  X[:, 0] -> input_ids / X[:, 1] -> token_type_ids / X[:, 2] -> attention_mask
    """
    encodings = tokenizer(text=sents,
                          # cls, sep 등의 토큰을 자동으로 추가
                          add_special_tokens=True,
                          # pytorch tensor로 출력
                          return_tensors='pt',
                          truncation=True,
                          padding=True)
    # input_ids: 각 토큰의 정수 인코딩의 나열
    # token_type_ids: 첫번째 문장 = 0, 두번재 문장=1의 나열
    # attention_mask: 어텐션값을 계산 = 1, 어텐션값을 계산 x = 0의 나열
    # 더 자세한 내용은 다음의 노트 참조:
    # BertTokenizer의 출력값에 대한 설명: https://www.notion.so/BERT-Why-what-and-how-d888343d82164e62a762cf9aa1186990#39064ae97c9a410980e4bab30ae0b610
    # 각 정수인코딩이 어떻게 사용되는가?: https://www.notion.so/BERT-Why-what-and-how-d888343d82164e62a762cf9aa1186990#ffde9fecdf9f4b6c9afb745d280769be
    return torch.stack([
        encodings['input_ids'],
        encodings['token_type_ids'],
        encodings['attention_mask']
    ], dim=1).to(device)

# 텐서를 구축하는 부분 - y
def build_y(labels: List[int], device: torch.device) -> torch.Tensor:
    return torch.FloatTensor(labels).unsqueeze(-1).to(device)



## Sentiment Classifier
![image](https://user-images.githubusercontent.com/56193069/136149922-6aca8615-7f1f-4cc6-80b8-8ab185230e11.png)


In [ ]:
class SentimentClassifier(torch.nn.Module):
    def __init__(self, bert: BertModel, device: torch.device):
        super().__init__()
        # --- define the layers to optmise --- #
        # hyper parameters
        self.H = bert.config.hidden_size
        # TODO 1
        self.bert = bert  # 사전학습된 버트도 최종 신경망의 일부분으로 정의.
        # 학습해야하는 가중치에는 무엇이 있는지 - 이걸 먼저 파악하고 정의 해야합니다.
        self.W_hy = torch.nn.Linear(self.H, 1)
        self.to(device)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X: (N, 3, L)
        :return: H_all (N, L, H)
        """
        input_ids = X[:, 0]
        token_type_ids = X[:, 1]
        attention_mask = X[:, 2]
        # --- bert의 맥락이 반영된 히든벡터 출력 --- # 
        H_all = self.bert(input_ids, token_type_ids, attention_mask)[0]
        return H_all

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X: (N, 3, L)
        :return: (N, 1)
        """
        # --- 긍정/부정 확률을 N개의 데이터에 대하여 예측 --- #
        # TODO 2
        H_all = self.forward(X)  # (N, 3, L) -> (N, L, H)
        # N개의 데이터가 긍정적일 확률을 출력하기
        H_cls = H_all[: , 0]  # (N, L, H) -> (N, H). list slicing.
        y_hat = self.W_hy(H_cls)  # (N, H) * (?=H, ?=1) -> (N, 1). matrix multiplication with W_hy
        y_hat_norm = torch.sigmoid(y_hat)  # (N, 1) -> (N, 1); [0-1]. torch.sigmoid() 
        return y_hat_norm

    def training_step(self, X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        """
        : param X: (N, 3, L)
        : y: (N, 1)
        :return: loss (1,)
        """
        # --- y, y_hat 사이의 로스를 계산해서 출력 --- #
        # TODO 3
        y_hat = self.predict(X)  # (N, 3, L) -> (N, 1)
        # N개의 데이터의 로스를 계산하기.
        loss = F.binary_cross_entropy(y_hat, y)  # (N, 1), (N, 1) -> (N, 1). F.binary_cross_entropy
        loss_sum = loss.sum()  # (N, 1) -> (1,). Tensor.sum()
        return loss_sum

## Analyser

In [ ]:

class Analyser:
    """
    BERT 기반 감성분석기.
    """
    def __init__(self, classifier: SentimentClassifier, tokenizer: BertTokenizer, device: torch.device):
        self.classifier = classifier
        self.tokenizer = tokenizer
        self.device = device

    def __call__(self, text: str) -> float:
        X = build_X(sents=[text], tokenizer=self.tokenizer, device=self.device)
        y_hat = self.classifier.predict(X)
        return y_hat.item()


## Training

In [ ]:
# 사전학습된 버트 모델을 로드
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
bert = BertModel.from_pretrained('beomi/kcbert-base')

# --- have a look at the config --- #
print(bert.config)
print(bert.config.hidden_size)

Downloading:   0%|          | 0.00/244k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

768


In [ ]:
# --- hyper parameters --- #
EPOCHS = 10
LR = 0.0001


device = load_device()
print(device)  # gpu 사용가능한지 확인하기

# --- build the dataset --- # 
sents = [sent for sent, _ in DATA]
labels = [label for _, label in DATA]
X = build_X(sents, tokenizer, device)
y = build_y(labels, device)

# --- instantiate the classifier --- #
classifier = SentimentClassifier(bert, device)
classifier.train() # 가중치 변경 허용
# 최적화 알고리즘 선택
optimizer = torch.optim.Adam(classifier.parameters(), lr=LR)

# --- 학습시작 --- #
for epoch in range(EPOCHS):
    loss = classifier.training_step(X, y)
    loss.backward()  # 오차 역전파
    optimizer.step()  # 경사도 하강
    optimizer.zero_grad()  # 기울기 축적방지
    print(f"epoch:{epoch}, loss:{loss.item()} ")


cuda
epoch:0, loss:0.41232842206954956 
epoch:1, loss:0.004942582920193672 
epoch:2, loss:0.0005476258229464293 
epoch:3, loss:0.00017030214075930417 
epoch:4, loss:0.00010694361117202789 
epoch:5, loss:7.961970550240949e-05 
epoch:6, loss:5.9162299294257537e-05 
epoch:7, loss:5.358939233701676e-05 
epoch:8, loss:4.316441118135117e-05 
epoch:9, loss:4.42890559497755e-05 


## Test

In [ ]:
TESTS = [
    "나는 자연어처리가 좋아",
    "나는 자연어처리가 싫어",
    "나는 너가 좋다",
    "너는 참 좋다",
    "오늘 날씨가 흐리네",
    "코로나19가 지긋지긋하다" # 전이학습의 장점을 보여주는 데이터
]

In [ ]:
classifier.eval()  # 가중치 변경 불허
analyser = Analyser(classifier, tokenizer, device)

for sent in TESTS:
    print(sent, "->", analyser(sent))

나는 자연어처리가 좋아 -> 0.999990701675415
나는 자연어처리가 싫어 -> 3.246697815484367e-05
나는 너가 좋다 -> 0.9999692440032959
너는 참 좋다 -> 0.9999889135360718
오늘 날씨가 흐리네 -> 2.5797542548389174e-05
코로나19가 지긋지긋하다 -> 3.483594991848804e-05
